In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%load_ext rpy2.ipython

In [3]:
%%R

install.packages("tidyr")
install.packages("dplyr")
install.packages("speedglm")
install.packages("boot")
install.packages("purr")
install.packages("readr")
install.packages("googledrive")

library(tidyr)
library(dplyr)
library(speedglm)
library(boot)
library(tidyverse)
library(readr)
library(googledrive)

(as ‘lib’ is unspecified)







	‘/tmp/RtmpdPmtDs/downloaded_packages’

(as ‘lib’ is unspecified)







	‘/tmp/RtmpdPmtDs/downloaded_packages’

(as ‘lib’ is unspecified)














	‘/tmp/RtmpdPmtDs/downloaded_packages’

(as ‘lib’ is unspecified)







	‘/tmp/RtmpdPmtDs/downloaded_packages’

(as ‘lib’ is unspecified)

(as ‘lib’ is unspecified)







	‘/tmp/RtmpdPmtDs/downloaded_packages’

(as ‘lib’ is unspecified)







	‘/tmp/RtmpdPmtDs/downloaded_packages’

Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union



Attaching package: ‘Matrix’



    expand, pack, unpack



Attaching package: ‘MASS’



    select




Attaching package: ‘boot’



    control




── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.4.4     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::%||%()    masks base::%||%()
✖ Matrix::expand() masks tidyr::expand()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ Matrix::pack()   masks tidyr::pack()
✖ MASS::select()   masks dplyr::select()
✖ Matrix::unpack() masks tidyr::unpack()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [4]:
%%R

wide_to_long <- function(df, z_var = "Z", y_var = "Y") {
  df %>% pivot_longer(cols = c("year1", "year2", "year3", "year4"),
                      values_to = y_var,
                      names_to = "year") %>%
    mutate(year = as.integer(stringr::str_sub(year, 5, 5))) %>%
    mutate(Z_year3 = as.numeric(Z == 1 & year == '3')) %>%
    mutate(Z_year4 = as.numeric(Z == 1 & year == '4')) %>%
    mutate(G = 3*Z)
}

### pivot a long data frame to a wide format
long_to_wide_practice <- function(df, keep_weights = T) {
  id_cols <- c("id_practice", "Z", "n_patients",
               "V1_avg", "V2_avg", "V3_avg", "V4_avg", "V5_A_avg", "V5_B_avg", "V5_C_avg",
               "X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8", "X9")
  if ("weights" %in% colnames(data) && keep_weights) id_cols <- c(id_cols, "ps", "weights")
  df %>%
    pivot_wider(id_cols = all_of(id_cols),
                values_from = c("Y"),
                names_from = "year",
                names_prefix = "year"
    )
}

In [5]:
%%R

setwd("/content/drive/My Drive/")

merge_files <- function(index) {
  practice_year_df <- read_csv(sprintf("practice_year/acic_practice_year_%04d.csv", index))
  practice_df <- read_csv(sprintf("practice/acic_practice_%04d.csv", index))

  merged_df <- practice_year_df %>%
  dplyr::filter(year == 2) %>%
  dplyr::select(id.practice, Z, n.patients, V1_avg, V2_avg, V3_avg, V4_avg,
                               V5_A_avg, V5_B_avg, V5_C_avg) %>%
  left_join(practice_year_df %>% dplyr::select(id.practice, year, Y, post), by = "id.practice") %>%
  left_join(practice_df, by = "id.practice")

  colnames(merged_df) <- gsub('\\.', '_', colnames(merged_df))


  merged_df <- long_to_wide_practice(merged_df)

  return(merged_df)
}

# Generate a list of merged data frames
merged_dfs <- purrr::map(1:3400, merge_files)



Streaming output truncated to the last 5000 lines.
chr (2): X2, X4
dbl (8): id.practice, X1, X3, X5, X6, X7, X8, X9

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2000 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (13): id.practice, year, Y, Z, post, n.patients, V1_avg, V2_avg, V3_avg,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 500 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): X2, X4
dbl (8): id.practice, X1, X3, X5, X6, X7, X8, X9

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2000 Columns: 13
── Column spec

In [6]:
%%R

add_ps2 <- function(dfw, z_var = "Z",
                    x_vars = c("V1_avg", "V2_avg", "V3_avg", "V4_avg",
                               "V5_A_avg", "V5_B_avg", "V5_C_avg",
                               "X1", "X2", "X3", "X4", "X5",
                               "X6", "X7", "X8", "X9"
                    ),
                    group_var = NA,
                    ps_var = "ps", weight_var = "weights",
                    method = "ps", estimand = "ATT", max_weight = Inf, ...) {
  require(speedglm)
  if (!is.na(group_var)) {
    fml <- paste(z_var, "~", group_var, "*(", paste(setdiff(x_vars, group_var), collapse = "+"), ")")
  } else {
    fml <- paste(z_var, "~", paste(x_vars, collapse = "+"))
  }

  m <- tryCatch(
    {
      cat("try speedglm\n")
      speedglm(as.formula(fml), data = dfw, family=binomial('logit'), fitted = TRUE)
    },
    error=function(cond) {
      return(NA)
    }
  )
  if (is.na(m[1])) {
    m <- tryCatch(
      {
        cat("try glm\n")
        stats::glm(as.formula(fml), data = dfw, family=binomial('logit'))
      },
      error=function(cond) {
        return(NA)
      }
    )
  }

  if (!is.na(m[1])) {
    ps <- fitted(m)
    ps <- pmin(ps, 1 - 1e-16)
    prop <- mean(as.numeric(pull(dfw,z_var)))
    weights <- pull(dfw,z_var) * 1 + (1-pull(dfw,z_var)) * ps/(1-ps) *(1-prop)/prop
    res <- data.frame(ps, pmin(weights, max_weight))
    colnames(res) <- c(ps_var, weight_var)
    attr(res, "vcov") <- vcov(m)
    attr(res, "formula") <- as.formula(fml)
  } else {
    cat("Fail to fit glm!", group_var, "\n")
    # use a existing ps
    if (weight_var %in% colnames(dfw)) {
      cat("Use the original data ps\n")
      # do nothing
      res <- dfw[,c(ps_var, weight_var)]
      colnames(res) <- c(ps_var, weight_var)
    } else if("ps" %in% colnames(dfw)) {
      cat("Use the overall ps\n")
      res <- dfw[,c("ps", "weights")]
      colnames(res) <- c(ps_var, weight_var)
    } else {
      stop("No existing ps to use!")
    }
  }

  return(res)
}

In [7]:
%%R

fit_model_ipwdid_practice <- function(dfw, dfl = NULL,
                             max_weight = Inf,
                             group_ps = FALSE,
                             method = "boot",
                             refit = FALSE,
                             boot_method = c("all", "norm", "basic", "perc"),
                             n_boot = 20, alpha = 0.1) {
  require(speedglm)
  require(boot)

  variables <- c("Overall", "X1", "X2", "X3", "X4", "X5")
  psdf <- add_ps2(dfw, max_weight = max_weight)
  dfw$ps <- psdf$ps
  dfw$weights <- psdf$weights
  boot <- method %in% c("boot", "all")
  if (boot_method == "all") {
    boot_type <- c("norm", "basic", "perc")
  } else {
    boot_type <- boot_method
  }

  if (boot && group_ps) {
    # create all the ps needed before bootstrapping
    for (var in setdiff(variables, "Overall")) {
      psdf <- add_ps2(dfw, z_var = "Z",
                      group_var = var,
                      ps_var = paste0("ps_", var),
                      weight_var = paste0("weights_", var),
                      max_weight = max_weight)
      dfw[,paste0("ps_", var)] <- psdf[,paste0("ps_", var)]
      dfw[,paste0("weights_", var)] <- psdf[,paste0("weights_", var)]
    }
  }

  final_res <- data.frame()

  if (boot) {
    t_boot_start <- proc.time()
    res <- lapply(variables, function(var) {
      level_list <- sort(unique(as.character(dfw[[var]])))
      if (var == "Overall") level_list <- NA
      lapply(level_list, function(level) {
        get_att_level_boot_practice(dfw, var = var, level = level, alpha = alpha, group_ps = group_ps, max_weight = max_weight)
      }) %>% do.call(rbind, .)
    }) %>% do.call(rbind, .)

    res_boot <- boot(
      data = dfw, R = n_boot, sim = "ordinary",
      statistic = function(boot_df, boot_ind) {
        boot_df <- boot_df[boot_ind,]
        res <- lapply(variables, function(var) {
          if (refit) {
            # refit pscore, otherwise use the overall score
            psdf <- add_ps2(boot_df, z_var = "Z",
                            group_var = var,
                            ps_var = paste0("ps_", var),
                            weight_var = paste0("weights_", var),
                            max_weight = max_weight)
            boot_df[,paste0("ps_", var)] <- psdf[,paste0("ps_", var)]
            boot_df[,paste0("weights_", var)] <- psdf[,paste0("weights_", var)]
          }

          level_list <- sort(unique(as.character(boot_df[[var]])))
          if (var == "Overall") level_list <- NA
          lapply(level_list, function(level) {
            get_att_level_boot_practice(boot_df, var = var, level = level, alpha = alpha, group_ps = group_ps, max_weight = max_weight)
          }) %>% do.call(rbind, .)
        }) %>% do.call(rbind, .)
        res$satt
      })

    res_boot_ci <- lapply(1:length(res_boot$t0), function(i) {
      ci <- boot.ci(res_boot, conf = 1-alpha, type = boot_type, index = i)
      r <- data.frame()
      for (type in boot_type) {
        pos = c(4, 5)
        if (type == "norm") {
          pos <- pos - 2
          type <- "normal"
        }
        if (type == "perc") {
          type <- "percent"
        }
        r <- rbind(r,
                   data.frame(
                     index = i,
                     ci_method = type,
                     lower90 = ci[[type]][pos[1]],
                     upper90 = ci[[type]][pos[2]]
                   )
        )
      }
      r
    }) %>% do.call(rbind, .)

    res$index <- 1:nrow(res)
    res <- res %>% left_join(res_boot_ci) %>% dplyr::select(-index)

    res_boot_bias <- colMeans(res_boot$t) - res_boot$t0
    res$bias <- res_boot_bias

    final_res <- rbind(final_res, res)
    t_boot_end <- proc.time()
    print(t_boot_end - t_boot_start)
  }
  final_res
}

get_att_level_boot_practice <- function(dfw, var = c("Overall", "X1", "X2", "X3", "X4", "X5"),
                               level = NA, alpha = 0.1, group_ps = TRUE, max_weight = Inf) {
  # var <- "X1"
  # var <- "Overall"
  # level <- "0"
  if (var == "Overall") {
    df <- dfw
  } else {
    df <- dfw %>% filter(!!as.name(var) == level)
  }

  if (group_ps && var != "Overall") {
    df[["ps"]] <- df[[paste0("ps_", var)]]
    df[["weights"]] <- df[[paste0("weights_", var)]]
  }

  res <- data.frame()
  if (var == "Overall") {
    y_list <- c(3, 4, NA)
  } else {
    y_list <- c(NA)
  }

  res <- lapply(y_list, function(y) {
    # y = 3
    if (!is.na(y)) {
      w.treat0 <- df$Z
      w.cont0 <- pmin(df$ps * (1 - df$Z)/(1 - df$ps), max_weight)
      deltaY <- df[[paste0("year", y)]] - df[["year2"]]
      w.patient <- df$n_patients
    } else {
      w.treat0 <- c(df$Z, df$Z)
      w.cont0 <- pmin(c(df$ps * (1 - df$Z)/(1 - df$ps), df$ps * (1 - df$Z)/(1 - df$ps)), max_weight)
      deltaY <- c(df[[paste0("year", 3)]] - df[["year2"]], df[[paste0("year", 4)]] - df[["year2"]])
      w.patient <- c(df$n_patients, df$n_patients)
    }

    ind <- which(!is.na(deltaY))
    w.treat <- w.treat0[ind]
    w.cont <- w.cont0[ind]
    deltaY <- deltaY[ind]

    att.treat <- w.treat * deltaY
    att.cont <- w.cont * deltaY
    eta.treat <- sum(att.treat * w.patient) / sum(w.treat * w.patient)
    eta.cont <- sum(att.cont * w.patient) / sum(w.cont * w.patient)
    ipw.att <- eta.treat - eta.cont

    data.frame(variable = var, level = level, year = y, satt = ipw.att
    )
  }) %>% do.call(rbind, .)

  res
}


In [9]:
%%R

results <- map(merged_dfs, ~fit_model_ipwdid_practice(.x, max_weight = 5, group_ps = FALSE, method = "boot", boot_method = "basic", n_boot = 20, refit = FALSE))

Streaming output truncated to the last 5000 lines.
Joining with `by = join_by(index)`
   user  system elapsed 
  1.342   0.101   1.367 
try speedglm
Joining with `by = join_by(index)`
   user  system elapsed 
  1.485   0.049   1.577 
try speedglm
Joining with `by = join_by(index)`
   user  system elapsed 
  1.477   0.050   1.564 
try speedglm
Joining with `by = join_by(index)`
   user  system elapsed 
  1.001   0.092   1.014 
try speedglm
Joining with `by = join_by(index)`
   user  system elapsed 
  0.991   0.050   0.995 
try speedglm
Joining with `by = join_by(index)`
   user  system elapsed 
  1.004   0.053   1.031 
try speedglm
Joining with `by = join_by(index)`
   user  system elapsed 
  1.011   0.047   1.008 
try speedglm
Joining with `by = join_by(index)`
   user  system elapsed 
  1.016   0.077   1.016 
try speedglm
Joining with `by = join_by(index)`
   user  system elapsed 
  1.030   0.089   1.051 
try speedglm
Joining with `by = join_by(index)`
   user  system elapsed 
  1.046

In [10]:
%%R

extracted_results <- map2(results, 1:3400, ~ .x %>%
                            filter(variable == "Overall") %>%
                            mutate(new_index = .y)) # Add the file index as a new column

# Combine all results into a single data frame
final_results <- bind_rows(extracted_results)

In [11]:
%%R

write_csv(final_results, "final_results.csv")